In [3]:
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten
from sklearn.metrics import accuracy_score
import tensorflow_datasets as tfds
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.models import Model
from sklearn.neighbors import NearestNeighbors

Data Processing

In [4]:
ds = tfds.load('lfw', split='train', as_supervised=True) #Get's the data in a tuple from tensroflow

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/lfw/0.1.0.incompleteCG49CQ/lfw-train.tfrecord


  0%|          | 0/13233 [00:00<?, ? examples/s]

Dataset lfw downloaded and prepared to /root/tensorflow_datasets/lfw/0.1.0. Subsequent calls will reuse this data.


In [5]:
labels = np.array(list(map(lambda x: x[0], ds))) #Maps the label in an array with each element being a tensors

In [6]:
labellist=[] #Makes a list of the labels with the elements being arrays now
for x in range(13233):
  labellist.append(np.array(labels[x]))

In [7]:
labellist=np.array(labellist) #Makes the list of labels an array

In [8]:
labelliststr=[] #Makes a list of labels with each element being a string now
for x in range(13233):
  labelliststr.append(str(labellist[x]))

In [9]:
labellistarr=np.array(labelliststr) #Makes the list of labels an array

In [10]:
from numpy import save #I saved the data so that I could run it from my computer as I ran out of ram on colab
np.save('labellistarr', labellistarr)

In [11]:
numericlabel = pd.get_dummies(labellistarr) #Had to get a numeric value for all of the labels, this made it in one-hot encoding form

In [12]:
endlabels=numericlabel.values.argmax(1)  #Got an array of numeric values where each name corresponds to a number

In [13]:
np.save('endlabels', endlabels) #Saved it to run from computer

In [14]:
images = np.array(list(map(lambda x: x[1], ds))) #Maps the images in an array with each element being an array of an image

In [15]:
np.save('images', images) #Saved it to run from computer

In [16]:
ramshort=np.unique(endlabels[0:297]) #Ran out of ram for all of the categories to be in the output layer, so I took a sample set with the all of the observations taht had the first 250 unique values as a label

In [17]:
ramshort.shape

(250,)

In [18]:
samplelabels=endlabels[0:297] #Grabs the first 297 values in the labels

In [19]:
samplelabels = pd.get_dummies(samplelabels) #One-hot encodes it

In [20]:
samplelabels=samplelabels.values.argmax(1) #Numeric values that corresponds to each unique name

In [21]:
from numpy import load #Test out the loaded data
numericlabels = load('/content/endlabels.npy')
imagedata = load('/content/images.npy')
namedlabels = load('/content/labellistarr.npy')

CNN

In [ ]:
model = Sequential( #This is for the actual complete data not the sample
    [
    Conv2D(32, (3,3), padding='valid', activation="relu", strides=1, input_shape=(250, 250, 3)),
    MaxPooling2D((2, 2), strides=2),

    Conv2D(64, (3,3), padding='valid', activation="relu"),
    MaxPooling2D((2, 2), strides=2),
        
    Conv2D(128, (3,3), padding='valid', activation="relu"),
    MaxPooling2D((2, 2), strides=2),

    Flatten(),
    Dense(5749, activation="softmax")
]
)

In [ ]:
model.compile(loss='sparse_categorical_crossentropy', optimizer="adam", metrics='accuracy') 

In [ ]:
model.fit(imagedata, numericlabels, epochs=10, batch_size=100) 

In [ ]:
model.save('C:/Users/poona/Downloads/ModelReal') #About 96% - Saved it - Might have to change the location

Load Model

In [ ]:
new_model = tf.keras.models.load_model('C:/Users/poona/Downloads/ModelReal') #Tested out the loaded model - Might have to change the location

In [ ]:
#new_model.fit(imagedata, numericlabels, epochs=1, batch_size=100) #See if the percents lined up

In [ ]:
ModelWOClass = Model(new_model.input, new_model.layers[-2].output) #Model without the classification layer

Similarity Search

In [ ]:
features = ModelWOClass.predict(imagedata) #Grabs the features from the model, when it flattens them

In [ ]:
np.save('features', features) #Saves the features after the images were sent trough the prediction model for easier access

In [ ]:
neighbors = NearestNeighbors(n_neighbors=5, algorithm='auto', metric='minkowski').fit(features) #K-nearest neighbors with the top 5 images closest to the query image, the first one is going to be the query image
#The algorithm is auto so it finds the best spacial partitioning for the data, in this case I believe they use brute force, so there isn't any weights attached to the spaciel divide between the features
#The metric is minkowski, which is the distance method between the features

In [ ]:
distances, indices = neighbors.kneighbors([features[13]]) #Pick the image you want to query as the number reference
#distances shows the differences between that query image and the images the algorithm brings back, while indices shows theh actual image number

In [ ]:
distances #Distances between query image and query results

In [ ]:
indices #Image references

In [ ]:
#Plot's first result from search - query image
plt.imshow(imagedata[indices[0,0]])
namedlabels[indices[0,0]]

In [ ]:
#Plot's second result from search
plt.imshow(imagedata[indices[0,1]])
namedlabels[indices[0,1]]

In [ ]:
#Plot's third result from search
plt.imshow(imagedata[indices[0,2]])
namedlabels[indices[0,2]]

In [ ]:
#Plot's fourth result from search
plt.imshow(imagedata[indices[0,3]])
namedlabels[indices[0,3]]

In [ ]:
#Plot's fifth result from search
plt.imshow(imagedata[indices[0,4]])
namedlabels[indices[0,4]]